In [1]:
import pandas as pd
from matplotlib import pyplot as plt
#!pip install pivottablejs
import seaborn as sns

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
latin = pd.read_stata('dataset/Latinobarometro_2020_Esp_Stata_v1_0.dta',convert_categoricals=False)
gpt = pd.read_csv('respuestas_gpt35_turbo.csv')
cohere = pd.read_csv('respuestas_cohere.csv')
bard = pd.read_csv('respuestas_bard.csv')
categorias = pd.read_csv('dataset/categorias_agrupadas.csv')

In [3]:
latin = latin[latin['idenpa']==32] #solo los argentinos

In [4]:
# Crea un StataReader para leer los metadatos
reader = pd.io.stata.StataReader('dataset/Latinobarometro_2020_Esp_Stata_v1_0.dta')

In [5]:
import pickle
with open(r"respuestas.pickle", "rb") as input_file:
    respuestas = pickle.load(input_file)
with open(r"preguntas_seleccionadas.pickle", "rb") as input_file:
    preguntas_seleccionadas = pickle.load(input_file)

In [6]:
categorias['pregunta_codigo'] = categorias['Latinobarometro'].apply(lambda x: x.split('-')[0])
categorias.head()

,Latinobarometro,encuesta,categorias,pregunta_codigo
0,"P11S0037- En general, ¿Diría Ud. que está muy ...",Latinobarometro,democracia - excesos poder ej - presidencialismo,P11S0037
1,"P11S0038- En general, ¿Diría Ud. que está muy ...",Latinobarometro,economia,P11S0038
2,P2ST- ¿Diría Ud. que Argentina en el año 2020 ...,Latinobarometro,relaciones internacionales,P2ST
3,P12ST- En términos generales ¿Diría usted que ...,Latinobarometro,democracia - excesos poder ej - presidencialismo,P12ST
4,P4STGBS- ¿Cómo calificaría en general la situa...,Latinobarometro,economia,P4STGBS


In [7]:
categorias['categorias'].value_counts()

relaciones internacionales                                                               16
opinion                                                                                  15
economia                                                                                 12
democracia - excesos poder ej - presidencialismo                                         11
ideologia politica - fuerzas armadas extranjeras - inmigracion extranjera - seguridad    11
derechos sociales- pobreza - ingresos - educacion                                        11
corrupcion - confianza en insitituciones                                                  3
justicia - independencia de poderes                                                       1
Name: categorias, dtype: int64

In [8]:
preguntas_topico = list(categorias[(categorias['categorias']=='relaciones internacionales')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P26N_A',
 'P30ST_A',
 'P30ST_B',
 'P30ST_C',
 'P30ST_D',
 'P30ST_E',
 'P30ST_F',
 'P31ST_A',
 'P31ST_B',
 'P31ST_C',
 'P31ST_D',
 'P31ST_E',
 'P25N',
 'P35N_A',
 'P35N_B']

In [39]:
preguntas_sin_orden = ['P33N','P34N']

In [40]:
import math
pregs = []
def calculate_distance(responde,llm, preguntas):    
    result = 0
    num_pregs = 0
    for pregunta in list(preguntas):
        if((pregunta.lower() in responde.keys()) and (not math.isnan(responde[pregunta.lower()])) and (pregunta in list(llm['Pregunta'])) 
           and(llm[llm['Pregunta']== pregunta]['Respuesta'].values[0] > 0) and(llm[llm['Pregunta']== pregunta]['Respuesta'].values[0] < 15) and (not pregunta in preguntas_sin_orden )):
            
            result = result + (abs(responde[pregunta.lower()]-llm[llm['Pregunta']== pregunta]['Respuesta'].values[0]))
            num_pregs = num_pregs + 1
            pregs.append(pregunta)
    if(num_pregs == 0):
        result = -1
        norm_result = -1
    else:
        norm_result = result/num_pregs
    
    return(result, num_pregs, norm_result)
calculate_distance(latin.loc[0],bard,preguntas_topico)

(8, 7, 1.1428571428571428)

In [10]:
gpt[gpt['Pregunta']=='P31ST_B']

,Unnamed: 0,Pregunta,Respuesta
44,44,P31ST_B,-2


In [11]:
# Supongamos que latin_distances es un DataFrame de pandas.

# Reemplazo "Ninguna", "No sabe", y "No contesta" en ideo por NA
latin['p18st'] = latin['p18st'].replace([97, -1, -2], None)

# Reemplazo "No sabe" y "No contesta" en interes politica por NA
latin['p46stgbs'] = latin['p46stgbs'].replace([-1, -2], None)

# No especificaste el reemplazo para 's13', pero si fuera similar a los anteriores, sería así:
latin['s13'] = latin['s13'].replace([-5], None)

df_clean = latin.dropna(subset=['p18st','p46stgbs','s13'])
df_clean.shape

(739, 408)

In [12]:
df_clean['p18st'] = df_clean['p18st'].astype(float)
df_clean['p46stgbs'] = df_clean['p46stgbs'].astype(float)
df_clean['s13'] = df_clean['s13'].astype(float)

/tmp/ipykernel_37400/1763769106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['p18st'] = df_clean['p18st'].astype(float)
/tmp/ipykernel_37400/1763769106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['p46stgbs'] = df_clean['p46stgbs'].astype(float)
/tmp/ipykernel_37400/1763769106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

# Relacione internacionales

In [14]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [15]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 412)

In [16]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     23.54
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           6.24e-29
Time:                        15:24:44   Log-Likelihood:                -2392.9
No. Observations:                 739   AIC:                             4802.
Df Residuals:                     731   BIC:                             4839.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.4106      2.049      5.081      0.0

## Bard

- Edad (adulto)
- Sexo (hombre)
- Ideología (derecha)
- Estudios (Más)
- Politizado(Más)

In [17]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [19]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(0, 416)

## GPT

- No hay suficientes casos

In [24]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be s

In [25]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(0, 420)

## Cohere

- No hay suficientes casos

# Economía

In [20]:
preguntas_topico = list(categorias[(categorias['categorias']=='economia')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P4STGBS',
 'P5STGBS',
 'P28ST',
 'P29ST_A',
 'P29ST_B',
 'P29ST_C',
 'P29ST_D',
 'P29ST_E',
 'P29ST_F',
 'P33N',
 'P34N']

In [21]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)


In [22]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 416)

In [23]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     10.94
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           3.52e-13
Time:                        15:28:05   Log-Likelihood:                -2243.5
No. Observations:                 739   AIC:                             4503.
Df Residuals:                     731   BIC:                             4540.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.8391      1.674      7.073      0.0

## Bard

- Edad (adulto)
- Sexo (hombre)
- Ideología (derecha)
- Estudios (Más)
- Irse del país (sí)
- Politizado(Más)

In [27]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)


In [28]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(0, 420)

## GPT

- No hay suficientes casos

In [29]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)


In [30]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(0, 420)

## Cohere

- No hay suficientes casos

# Democracia - excesos poder ej - presidencialismo

In [31]:
preguntas_topico = list(categorias[(categorias['categorias']=='democracia - excesos poder ej - presidencialismo')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P12ST',
 'P14GBS',
 'P20ST_A',
 'P20ST_B',
 'P20ST_C',
 'P20ST_D',
 'P20ST_E',
 'P21STM',
 'P47ST_A',
 'P47ST_B']

In [32]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)


In [33]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 420)

In [34]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     11.84
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           2.44e-14
Time:                        15:40:02   Log-Likelihood:                -1830.3
No. Observations:                 739   AIC:                             3677.
Df Residuals:                     731   BIC:                             3713.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4465      0.957      5.692      0.0

## Bard

- Ideología (izquierda)
- Irse del país (no)
- Politizado(Más)

In [35]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)


In [36]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(0, 420)

## GPT

- No hay suficientes casos

In [37]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)


In [38]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(0, 420)

## Cohere

- No hay suficientes casos

# Ideologia politica - fuerzas armadas extranjeras - inmigracion extranjera - seguridad

In [42]:
preguntas_topico = list(categorias[(categorias['categorias']=='ideologia politica - fuerzas armadas extranjeras - inmigracion extranjera - seguridad')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P36N_A',
 'P36N_B',
 'P42N',
 'P37N_A',
 'P37N_B',
 'P37N_C',
 'P37N_D',
 'P47ST_J',
 'P47ST_K',
 'P49STGBS']

In [43]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)


In [44]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 420)

In [45]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     3.197
Date:                Fri, 14 Jul 2023   Prob (F-statistic):            0.00242
Time:                        15:49:16   Log-Likelihood:                -2392.4
No. Observations:                 739   AIC:                             4801.
Df Residuals:                     731   BIC:                             4838.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.3941      2.047      4.100      0.0

## Bard

- Sexo (Hombre)
- Irse del país (Sí)

In [56]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)


In [57]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(0, 420)

## GPT

- No hay suficientes casos

In [52]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)


In [53]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(0, 420)

## Cohere

- No hay suficientes casos

# Opinión

In [58]:
preguntas_topico = list(categorias[(categorias['categorias']=='opinion')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P10STGBS',
 'P17STGBS',
 'P22ST_A',
 'P22ST_B',
 'P22ST_C',
 'P22ST_D',
 'P23N',
 'P41N',
 'P46STGBS',
 'P47ST_C',
 'P47ST_D',
 'P48ST',
 'P60N_B',
 'P69ST']

In [59]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)


In [60]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 420)

In [61]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.096
Method:                 Least Squares   F-statistic:                     12.20
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           8.50e-15
Time:                        15:52:52   Log-Likelihood:                -1997.1
No. Observations:                 739   AIC:                             4010.
Df Residuals:                     731   BIC:                             4047.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.2679      1.199      8.563      0.0

## Bard

- Irse del pais (no)
- Estudios de los padres (Mas)
- Politizado

In [62]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)


In [63]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(0, 420)

## GPT

- No hay suficientes casos

In [64]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)


In [65]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(0, 420)

## Cohere

- No hay suficientes casos

# derechos sociales- pobreza - ingresos - educacion

In [66]:
preguntas_topico = list(categorias[(categorias['categorias']=='derechos sociales- pobreza - ingresos - educacion')  ]['pregunta_codigo'].values)
preguntas_topico[1:20]

['P19ST_A',
 'P29ST_G',
 'P29ST_H',
 'P47ST_E',
 'P47ST_F',
 'P47ST_G',
 'P47ST_H',
 'P47ST_I',
 'P47ST_L',
 'P47ST_M']

In [67]:
df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/787502377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bard_results'] = df_clean.apply(lambda x: calculate_distance(x, bard,preguntas_topico), axis=1)
/tmp/ipykernel_37400/787502377.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['bard_distance', 'bard_num_preg','bard_distance_norm']] = pd.DataFrame(df_clean['bard_results'].tolist(), index=df_clean.index)


In [68]:
df_clean[df_clean['bard_num_preg'] > 5].shape

(739, 420)

In [69]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['bard_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['bard_num_preg'] > 5]['bard_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          bard_distance   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.703
Date:                Fri, 14 Jul 2023   Prob (F-statistic):              0.105
Time:                        15:55:35   Log-Likelihood:                -2069.2
No. Observations:                 739   AIC:                             4154.
Df Residuals:                     731   BIC:                             4191.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.9792      1.322      6.035      0.0

## Bard

- Politizado

In [70]:
df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2116707411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['gpt_results'] = df_clean.apply(lambda x: calculate_distance(x, gpt,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2116707411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['gpt_distance', 'gpt_num_preg','gpt_distance_norm']] = pd.DataFrame(df_clean['gpt_results'].tolist(), index=df_clean.index)


In [71]:
df_clean[df_clean['gpt_num_preg'] > 5].shape

(739, 420)

In [75]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['gpt_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['gpt_num_preg'] > 5]['gpt_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           gpt_distance   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     6.271
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           3.69e-07
Time:                        15:56:34   Log-Likelihood:                -1965.9
No. Observations:                 739   AIC:                             3948.
Df Residuals:                     731   BIC:                             3985.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.7973      1.150      7.652      0.0

## GPT

- Edad (adulto)
- Politizado

In [72]:
df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)

/tmp/ipykernel_37400/2733857388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['cohere_results'] = df_clean.apply(lambda x: calculate_distance(x, cohere,preguntas_topico), axis=1)
/tmp/ipykernel_37400/2733857388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[['cohere_distance', 'cohere_num_preg','cohere_distance_norm']] = pd.DataFrame(df_clean['cohere_results'].tolist(), index=df_clean.index)


In [73]:
df_clean[df_clean['cohere_num_preg'] > 5].shape

(739, 420)

In [76]:
# Importando las bibliotecas necesarias
import statsmodels.api as sm
import pandas as pd

# Supongamos que tienes un DataFrame de pandas llamado df con 'Y' como tu variable dependiente 
# y 'X1', 'X2', 'X3' como tus variables independientes.

X = df_clean[df_clean['cohere_num_preg'] > 5][['edad','sexo', 'p18st' ,'s16','s13','s17','p46stgbs']] # estas son las variables independientes
y = df_clean[df_clean['cohere_num_preg'] > 5]['cohere_distance'] # esta es la variable dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) 

# Ajustar el modelo
model = sm.OLS(y, X)
results = model.fit()

# Imprimir los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        cohere_distance   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     4.977
Date:                Fri, 14 Jul 2023   Prob (F-statistic):           1.61e-05
Time:                        15:57:10   Log-Likelihood:                -1952.6
No. Observations:                 739   AIC:                             3921.
Df Residuals:                     731   BIC:                             3958.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.8095      1.129      8.687      0.0

## Cohere

- Edad (adulto)
- Irse del país (no)
- Politizado